In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import csv

# --- 設定 ---
# CSVデータ用の保存先パス
DATA_PATH_CSV = os.path.join(os.getcwd(), 'MP_Data_CSV') 
actions = np.array(['ageru', 'understand', 'annsinnsuru' , 'heavy'])
no_videos = 150
sequence_length = 30

# ★変更点: MediaPipe Handsモデルの準備
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def multiple_detection(image, model):
    """
    画像から手やポーズを検出する関数（この関数は変更なし）
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_hand_landmarks(image, results):
    """
    ★変更点: 検出した手のランドマークを描画する関数
    """
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS, # 手の骨格を描画
                mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
            )

# --- データの収集 ---
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

# ★変更点: mp_pose.Pose を mp_hands.Hands に変更
# max_num_hands=2 で両手を検出するように設定
with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands_model:
    # ★変更点: CSVのヘッダー行を両手用に動的に作成
    header = ['frame']
    # 左右の手(2) * 21ランドマーク * 3座標(x,y,z) のヘッダーを作成
    for hand_label in ['left', 'right']:
        for i in range(21):
            header += [f'{hand_label}_{i}_x', f'{hand_label}_{i}_y', f'{hand_label}_{i}_z']
    
    for action in actions:
        # CSV用のアクションごとのフォルダを作成
        action_path = os.path.join(DATA_PATH_CSV, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r'C:\Users\admin\Downloads\syuwavideo', f'{action}{video_num}.mp4')
            
            cap = cv2.VideoCapture(video_file_path)
            if not cap.isOpened():
                print(f"警告: 動画ファイルが開けませんでした: {video_file_path}")
                continue

            # 動画ごとにCSVファイルを開き、書き込み準備
            csv_file_path = os.path.join(action_path, f'{video_num}.csv')
            print(f'処理中の動画: {video_file_path}')
            
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow(header)  # ヘッダーを書き込む

                for frame_num in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        print(f"情報: 動画 {video_file_path} が {sequence_length} フレーム未満で終了しました。収集済みフレーム数: {frame_num}")
                        break

                    # ★変更点: hands_model を使って検出
                    image, results = multiple_detection(frame, hands_model)
                    # ★変更点: 新しい描画関数を呼び出し
                    draw_hand_landmarks(image, results)

                    cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                                (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                    # ★変更点: 両手のキーポイントを抽出し、CSVの1行として書き込む
                    # 126 = 2(左右) * 21(ランドマーク数) * 3(xyz座標)
                    row_data = [''] * 126 
                    
                    if results.multi_hand_landmarks and results.multi_handedness:
                        # 検出された各手についてループ
                        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                            # handednessから 'Left' or 'Right' を取得
                            hand_label = handedness.classification[0].label.lower()
                            
                            # 抽出したランドマークを一時リストに格納
                            hand_keypoints = []
                            for landmark in hand_landmarks.landmark:
                                hand_keypoints.extend([landmark.x, landmark.y, landmark.z])

                            # 右手か左手かに応じて、書き込む位置を決定
                            if hand_label == 'left':
                                row_data[0:63] = hand_keypoints
                            elif hand_label == 'right':
                                row_data[63:126] = hand_keypoints
                    
                    # フレーム番号とキーポイントデータを結合してCSVに書き込み
                    csv_writer.writerow([frame_num] + row_data)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                
            cap.release()
            print(f"データ保存完了: {csv_file_path}")

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

処理中の動画: C:\Users\admin\Downloads\syuwavideo\ageru1.mp4
情報: 動画 C:\Users\admin\Downloads\syuwavideo\ageru1.mp4 が 30 フレーム未満で終了しました。収集済みフレーム数: 23
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\ageru\1.csv
処理中の動画: C:\Users\admin\Downloads\syuwavideo\ageru2.mp4
情報: 動画 C:\Users\admin\Downloads\syuwavideo\ageru2.mp4 が 30 フレーム未満で終了しました。収集済みフレーム数: 25
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\ageru\2.csv
処理中の動画: C:\Users\admin\Downloads\syuwavideo\ageru3.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\ageru\3.csv
処理中の動画: C:\Users\admin\Downloads\syuwavideo\ageru4.mp4
データ保存完了: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_CSV\ageru\4.csv
処理中の動画: C:\Users\admin\Downloads\syuwavideo\ageru5.mp4
情報: 動画 C:\Users\admin\Downloads\syuwavideo\ageru5.mp4 が 30 フレーム未満で終了しまし